In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [2]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load_and_split()

In [3]:
def create_vector_store(_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(_docs)
    persist_directory = "./chroma_db"
    vectorstore = Chroma.from_documents(
        split_docs, 
        OpenAIEmbeddings(model='text-embedding-3-small'),
        persist_directory=persist_directory
    )
    return vectorstore

In [4]:
def add_to_vector_store(new_docs, persist_directory="./chroma_db"):
    # 1. 기존 벡터 저장소 불러오기
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(new_docs)

    # 3. 문서 추가
    vectorstore.add_documents(split_docs)

    return vectorstore

In [5]:
path='../data/문형빈 교수님 sci'
pl=os.listdir(path)

In [6]:
mk=load_and_split_pdf(os.path.join(path, pl[0]))
create_vector_store(mk)

In [7]:
for i in pl[1:]:
    ad=load_and_split_pdf(os.path.join(path, i))
    add_to_vector_store(ad)

In [9]:
# load from disk
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "HyungBin Moon's 2021 paper?"
result = db3.similarity_search(query)
print(result[0].page_content)

levels.Int J Energy Res. 2020;44:8037–8052. https://
doi.org/10.1002/er.4974
8052 MOON AND WOO
 1099114x, 2020, 10, Downloaded from https://onlinelibrary.wiley.com/doi/10.1002/er.4974 by Pukyong National University, Wiley Online Library on [14/04/2025]. See the Terms and Conditions (https://onlinelibrary.wiley.com/terms-and-conditions) on Wiley Online Library for rules of use; OA articles are governed by the applicable Creative Commons License


In [11]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "explain hyungbin moon's 2020 paper"
docs = db3.similarity_search(query, k=15)

formatted_docs = [Document(page_content=doc.page_content) for doc in docs]

prompt_template = """
다음은 논문 일부입니다. 이 내용을 바탕으로 석사 논문 내용을 연구대상 및 방법, 연구방법, 연구결과, 제언을 핵심적으로 요약해 주세요.
논문 내용:
{context}
요약:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context"])
llm = ChatOpenAI(model="gpt-4o")
chain = LLMChain(llm=llm, prompt=PROMPT)
context_text = "\n\n".join([doc.page_content for doc in formatted_docs])
summary = chain.run(context=context_text)
print(summary, "\n")

C:\Users\wonhy\AppData\Local\Temp\ipykernel_20212\2887330060.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db3 = Chroma(
C:\Users\wonhy\AppData\Local\Temp\ipykernel_20212\2887330060.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")
C:\Users\wonhy\AppData\Local\Temp\ipykernel_20212\2887330060.py:20: LangChainDeprecationWarning: The class `LLMChain` was depreca

**연구대상 및 방법:**
이 연구는 한국에서의 사용후핵연료 관리에 대한 대중의 태도와 선호를 분석하는 것을 목표로 하였습니다. 이를 위해 2020년 1월에 전문 설문 조사 회사인 마크로밀 엠브레인을 통해 온라인 설문 조사를 실시하였으며, 조사 대상은 20세에서 59세 사이의 성인 1000명으로, 성별, 연령, 지역 등 인구통계학적 특성을 고려한 비례층화표본추출 방법을 통해 선정되었습니다.

**연구방법:**
연구는 사용후핵연료 관리 전략에 대한 대중의 개별 및 세그먼트 수준에서의 선호도를 분석하였습니다. 이를 위해 다변량 분석 기법을 사용하여 다양한 관리 전략에 대한 대중의 수용도를 측정하였고, 각 전략의 속성에 대한 대중의 태도를 파악하였습니다.

**연구결과:**
연구 결과, 사용후핵연료 관리에 대한 대중의 수용도는 전략의 속성에 크게 의존하며, 특히 안전과 관련된 요소에 민감한 것으로 나타났습니다. 대중은 사용후핵연료의 위험성과 비용 증가에 대해 우려를 나타내었으며, 민주적 정책 과정이 반영된 관리 전략에 대해 더 높은 수용도를 보였습니다.

**제언:**
사용후핵연료 관리 정책을 효과적으로 수립하기 위해서는 대중의 안전에 대한 우려를 충분히 이해하고 이를 반영하는 정책 개발이 필요합니다. 또한, 정책 수립 과정에서 대중의 참여를 보장하고, 민주적 절차를 강화함으로써 정책의 수용도를 높일 수 있을 것입니다. 이러한 접근은 대중의 신뢰를 구축하고, 정책 실행의 원활함을 보장할 수 있는 기반이 될 것입니다. 

